In [2]:
from Bio import Entrez, SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import numpy as np
import pandas as pd
import re

import AS_motif_analysis as AS

In [4]:
# input significant alternative splicing (Skipped exons) evants with splicing sites coodinate
ri_sig = pd.read_csv('/home/yu/R_studio_work_dir/AS_further_analysis/output/RI_sig.csv')
print(ri_sig.iloc[:5,])

     ID           GeneID geneSymbol    chr strand  riExonStart_0base  \
0  2044  ENSG00000166946    CCNDBP1  chr15      +           43194080   
1   389  ENSG00000157353        FUK  chr16      +           70466881   
2  5260  ENSG00000168488     ATXN2L  chr16      +           28835932   
3  2049  ENSG00000143224       PPOX   chr1      +          161167099   
4  4203  ENSG00000231711  LINC00899  chr22      -           46042573   

   riExonEnd  upstreamES  upstreamEE  downstreamES   ...     \
0   43194461    43194080    43194137      43194414   ...      
1   70467966    70466881    70467471      70467885   ...      
2   28837237    28835932    28836485      28836727   ...      
3  161167486   161167099   161167234     161167338   ...      
4   46043699    46042573    46042640      46043562   ...      

   IncLevelDifference  IncLevel1-1 IncLevel1-2 IncLevel1-3 IncLevel1-4  \
0               0.645        1.000       1.000       1.000       0.778   
1              -0.302        0.000      

In [7]:
## Calculate the length of upstream,dowstream exon and intron, and flanked exon, 
## and calcualte the genome coodinates in order to download from NCBI
ri_sig_as_coord_len = pd.DataFrame(columns = ['ID','GeneID','geneSymbol','chr','strand',
                                              'seq_1_ex_co1','seq_1_ex_co2','seq_1_ex_co3','seq_1_ex_co4',
                                              'seq_2_ex_co1','seq_2_ex_co2','seq_2_ex_co3','seq_2_ex_co4'])

for i in range(0,len(ri_sig)):
    
    if ri_sig['strand'][i] == '+':
        ri_sig_as_coord_len.loc[i,'seq_1_ex_co2'] = ri_sig.loc[i,'upstreamEE'].copy()
        ri_sig_as_coord_len.loc[i,'seq_1_ex_co3'] = ri_sig.loc[i,'upstreamEE'].copy() + 1
        ri_sig_as_coord_len.loc[i,'seq_2_ex_co2'] = ri_sig.loc[i,'downstreamES'].copy() - 1
        ri_sig_as_coord_len.loc[i,'seq_2_ex_co3'] = ri_sig.loc[i,'downstreamES'].copy()
  
        if (ri_sig.loc[i,'upstreamEE'] - ri_sig.loc[i,'upstreamES'] + 1) >= 150:
            ri_sig_as_coord_len.loc[i,'seq_1_ex_co1'] = ri_sig.loc[i,'upstreamEE'].copy() - 150 + 1                     
        else:
            ri_sig_as_coord_len.loc[i,'seq_1_ex_co1'] = ri_sig.loc[i,'upstreamES']
                                                      
        if (ri_sig.loc[i,'downstreamES'] - ri_sig.loc[i,'upstreamEE'] - 1) >= 300:
            ri_sig_as_coord_len.loc[i,'seq_1_ex_co4'] = ri_sig.loc[i,'upstreamEE'] + 150
            ri_sig_as_coord_len.loc[i,'seq_2_ex_co1'] = ri_sig.loc[i,'downstreamES'] - 150                       
        else:
            ri_sig_as_coord_len.loc[i,'seq_1_ex_co4'] = ri_sig.loc[i,'upstreamEE'] + int((ri_sig.loc[i,'downstreamES'] - ri_sig.loc[i,'upstreamEE'] - 1)/2)
            ri_sig_as_coord_len.loc[i,'seq_2_ex_co1'] = ri_sig.loc[i,'downstreamES'] - int((ri_sig.loc[i,'downstreamES'] - ri_sig.loc[i,'upstreamEE'] - 1)/2)
        
        if (ri_sig.loc[i,'downstreamEE'] - ri_sig.loc[i,'downstreamES'] + 1) >= 150:
            ri_sig_as_coord_len.loc[i,'seq_2_ex_co4'] = ri_sig.loc[i,'downstreamES'] + 150 -1
        else:
            ri_sig_as_coord_len.loc[i,'seq_2_ex_co4'] = ri_sig.loc[i,'downstreamEE']
            
    else:

        ri_sig_as_coord_len.loc[i,'seq_1_ex_co2'] = ri_sig.loc[i,'downstreamES'].copy()
        ri_sig_as_coord_len.loc[i,'seq_1_ex_co3'] = ri_sig.loc[i,'downstreamES'].copy() - 1
        ri_sig_as_coord_len.loc[i,'seq_2_ex_co2'] = ri_sig.loc[i,'upstreamEE'].copy() + 1
        ri_sig_as_coord_len.loc[i,'seq_2_ex_co3'] = ri_sig.loc[i,'upstreamEE'].copy()
        
        if (ri_sig.loc[i,'downstreamEE'] - ri_sig.loc[i,'downstreamES'] + 1) >= 150:
            ri_sig_as_coord_len.loc[i,'seq_1_ex_co1'] = ri_sig.loc[i,'downstreamES'].copy() + 150 - 1        
        else:
            ri_sig_as_coord_len.loc[i,'seq_1_ex_co1'] = ri_sig.loc[i,'downstreamEE']
            
                                   
        if (ri_sig.loc[i,'downstreamES'] - ri_sig.loc[i,'upstreamEE'] - 1) >= 300:
            ri_sig_as_coord_len.loc[i,'seq_1_ex_co4'] = ri_sig.loc[i,'downstreamES'] - 150
            ri_sig_as_coord_len.loc[i,'seq_2_ex_co1'] = ri_sig.loc[i,'upstreamEE'] + 150                       
        else:
            ri_sig_as_coord_len.loc[i,'seq_1_ex_co4'] = ri_sig.loc[i,'downstreamES'] - int((ri_sig.loc[i,'downstreamES'] - ri_sig.loc[i,'upstreamEE'] - 1)/2)
            ri_sig_as_coord_len.loc[i,'seq_2_ex_co1'] = ri_sig.loc[i,'upstreamEE'] + int((ri_sig.loc[i,'downstreamES'] - ri_sig.loc[i,'upstreamEE'] - 1)/2)
        
        
        if (ri_sig.loc[i,'upstreamEE'] - ri_sig.loc[i,'upstreamES'] + 1) >= 150:
            ri_sig_as_coord_len.loc[i,'seq_2_ex_co4'] = ri_sig.loc[i,'upstreamEE'] -150 +1
        else:
            ri_sig_as_coord_len.loc[i,'seq_2_ex_co4'] = ri_sig.loc[i,'upstreamES']
                                   

ri_sig_as_coord_len.iloc[:,:5] = ri_sig.iloc[:,:5].copy()
ri_sig_as_coord_len

,ID,GeneID,geneSymbol,chr,strand,seq_1_ex_co1,seq_1_ex_co2,seq_1_ex_co3,seq_1_ex_co4,seq_2_ex_co1,seq_2_ex_co2,seq_2_ex_co3,seq_2_ex_co4
0,2044,ENSG00000166946,CCNDBP1,chr15,+,43194080,43194137,43194138,43194275,43194276,43194413,43194414,43194461
1,389,ENSG00000157353,FUK,chr16,+,70467322,70467471,70467472,70467621,70467735,70467884,70467885,70467966
2,5260,ENSG00000168488,ATXN2L,chr16,+,28836336,28836485,28836486,28836605,28836607,28836726,28836727,28836876
3,2049,ENSG00000143224,PPOX,chr1,+,161167099,161167234,161167235,161167285,161167287,161167337,161167338,161167486
4,4203,ENSG00000231711,LINC00899,chr22,-,46043699,46043562,46043561,46043412,46042790,46042641,46042640,46042573
5,1825,ENSG00000008710,PKD1,chr16,-,2097984,2097867,2097866,2097823,2097821,2097778,2097777,2097727
6,1644,ENSG00000179532,DNHD1,chr11,+,6558148,6558297,6558298,6558390,6558391,6558483,6558484,6558633
7,1955,ENSG00000111653,ING4,chr12,-,6652770,6652661,6652660,6652511,6652459,6652310,6652309,6652270
8,3635,ENSG00000132275,RRP8,chr11,-,6601297,6601148,6601147,6601102,6601101,6601056,6601055,6600925
9,3117,ENSG00000197114,ZGPAT,chr20,+,63733586,63733712,63733713,63733862,63734554,63734703,63734704,63734824
